# Yahtzee AI

## Rules of Yahtzee

Roll 5 dice. Any number of them can be rerolled twice, so 3 times total. Then fill one box corresponding to table below, either with the respective points, or with 0 if one doesn't want to or is not able to do otherwise.

| Shape                 | What to score        | Points |
| --------------------- | -------------------- | ------ |
| Ones                  | Total of Ones only   |        |
| Twos                  | Total of Twos only   |        |
| Threes                | Total of Threes only |        |
| Fours                 | Total of Fours only  |        |
| Fives                 | Total of Fives only  |        |
| Sixes                 | Total of Sixes only  |        |
| If 63+ points above   | 35 point bonus       |        |
| 3 of a Kind           | Total of all 5 dice  |        |
| 4 of a Kind           | Total of all 5 dice  |        |
| Full House            | 25 points            |        |
| Small Straight        | 30 points            |        |
| Large Straight        | 40 points            |        |
| Yahtzee (5 of a kind) | 50 points            |        |
| Chance                | Total of all 5 dice  |        |

If a minimum of 63 points are achieved in upper section, a 35 point bonus is issued.

If one has already rolled a Yahtzee and does so again, they get a 100-point bonus and must score the total of all 5 dice in the correcsponding box in the upper section. If that is already filled in, they can score in any open lower section box with the respective counting rules. If those are all filled, they must enter a zero somewhere in the upper section.

If one roles a Yahtzee, but has already entered zero in the Yahtzee box, they do not get a bonus, but can still enter it according to the order described above.

The game ends when all 13 boxes are filled, at which point the score is summed up, the more the better.

Detailed rules can be found under:
https://www.hasbro.com/common/instruct/Yahtzee.pdf

# Thoughts on complexity

## Number of possible game states:

Ones-Sixes: Empty, Zero, 5 additional score options

3/4 of a kind: Empty, Zero, 26 additional score options

Full House, Small/Large Straight, Yahtze: Empty, Zero, 1 score option

Chance: Empty, Zero, 26 score options

Plus 6 possible values for occurences of all 6 numbers in dice role.

And 3 options for rolls left (2, 1, none)

All in all $7^6 \cdot 28^2 \cdot 3^4 \cdot 28 \cdot 6^6 \cdot 3 = ~3x10^{16}$ possible states

In [1]:
import numpy as np
from numpy.random import default_rng

In [11]:
# Rolls a specified number of dice and returns them as an array
def dice_roll(number):
    # set up random number generator
    rng = default_rng()
    roll = rng.integers(1, 7, size=5)
    return roll

In [12]:
## Score Card ##
# Index 0 # Ones #
# Index 1 # Twos #
# Index 2 # Threes #
# Index 3 # Fours #
# Index 4 # Fives #
# Index 5 # Sixes #
# Index 6 # 3 of a Kind #
# Index 7 # 4 of a Kind #
# Index 8 # Full House #
# Index 9 # Small Straight #
# Index 10 # large Straight #
# Index 11 # Yahtzee #
# Index 12 # Chance #

# Initializes score card with -1, representing empty
def init_score_card():
    return np.full(13, -1)

In [93]:
### checks for the special throws one can make ###

def three_of_a_kind(dice):
    for i in range(3):
        if (dice==dice[i]).sum() >= 3:
            return True
    return False
    
def four_of_a_kind(dice):
    for i in range(2):
        if (dice==dice[i]).sum() >= 4:
            return True
    return False
    
def full_house(dice):
    first_number = dice[0]
    first_sum = (dice==first_number).sum()
    if first_sum < 2:
        return False
    for i in range(4):
        if dice[i+1] != first_number:
            second_number = dice[i+1]
            second_sum = (dice==second_number).sum()
            if second_sum == 2 and first_sum == 3:
                return True
            elif second_sum == 3 and first_sum == 2:
                return True
            else:
                return False
    return False
    
def small_straight(dice):
    sorted_dice = np.unique(dice)
    counter = 0
    dice_len = len(sorted_dice)
    if (dice_len < 4):
        return False
    for i in range(dice_len-1):
        if sorted_dice[i]+1==sorted_dice[i+1]:
            counter += 1
        else:
            counter = 0
    if counter >= 3:
        return True
    else:
        return False
    
def large_straight(dice):
    sorted_dice = np.unique(dice)
    if len(sorted_dice) < 5:
        return False
    for i in range(4):
        if sorted_dice[i]+1!=sorted_dice[i+1]:
            return False
    return True
    
def yahtzee(dice):
    if (dice==dice[0]).sum() == 5:
        return True
    else:
        return False

In [71]:
# Enter move as number between 0 and 12
# Computes and returns new score card state and score given
def make_move(state, move, dice):
    score = 0
    if move < 6:
        score = count_dice(move, dice)
    elif move == 6 and three_of_a_kind(dice):
        score = count_dice(-1, dice)
    elif move == 7 and four_of_a_kind(dice):
        score = count_dice(-1, dice)
    elif move == 8 and full_house(dice):
        score = 25
    elif move == 9 and small_straight(dice):
        score = 30
    elif move == 10 and large_straight(dice):
        score = 40
    elif move == 11 and yahtzee(dice):
        # here special rules need to be added
        score = 50
    elif move == 12:
        score = count_dice(-1, dice)
    state[move] = score

    return state, score

In [64]:
def count_dice(move, dice):
    if move == -1:
        return np.sum(dice)
    else:
        return (dice == move+1).sum()*(move+1)

In [96]:
### main game ###

score_card = init_score_card()
print('Ones, Twos, Threes, Fours, Fives, Sixes, Three_of_a_Kind, Four_of_a_Kind, Full_House, Small_Straight, Large_Straight, Yahtzee, Chance')
print('')



for j in range(100):
    dice = dice_roll(5)
    print('dice: ', dice)
    for i in range(13):
        make_move(score_card, i, dice)
    print('card after moves: ',score_card)
    print('')


Ones, Twos, Threes, Fours, Fives, Sixes, Three_of_a_Kind, Four_of_a_Kind, Full_House, Small_Straight, Large_Straight, Yahtzee, Chance

dice:  [4 5 6 3 1]
card after moves:  [ 1  0  3  4  5  6  0  0  0 30  0  0 19]

dice:  [2 6 1 5 5]
card after moves:  [ 1  2  0  0 10  6  0  0  0  0  0  0 19]

dice:  [1 2 3 4 5]
card after moves:  [ 1  2  3  4  5  0  0  0  0 30 40  0 15]

dice:  [1 1 5 2 2]
card after moves:  [ 2  4  0  0  5  0  0  0  0  0  0  0 11]

dice:  [3 5 5 1 1]
card after moves:  [ 2  0  3  0 10  0  0  0  0  0  0  0 15]

dice:  [6 1 6 3 4]
card after moves:  [ 1  0  3  4  0 12  0  0  0  0  0  0 20]

dice:  [3 2 4 1 2]
card after moves:  [ 1  4  3  4  0  0  0  0  0 30  0  0 12]

dice:  [5 3 3 4 5]
card after moves:  [ 0  0  6  4 10  0  0  0  0  0  0  0 20]

dice:  [1 6 3 4 4]
card after moves:  [ 1  0  3  8  0  6  0  0  0  0  0  0 18]

dice:  [2 4 4 2 6]
card after moves:  [ 0  4  0  8  0  6  0  0  0  0  0  0 18]

dice:  [5 1 6 1 3]
card after moves:  [ 2  0  3  0  5  6  0  0  0